In [81]:
%pip install nltk


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [82]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize

In [83]:
## First training a unigram model
## we currently have a dataset of sentences (train_set.csv)
## create a vocabulary of words(which contains unique words, here we are excluding the start and end of the sentence(/s and /*s) because we won't need in case of unigram model)

## prepoccesing also included in this step 

def vocab() :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    comm=test_set['0']
    ## creating a list of words 
    all_words=[]
    for comment in comm :
        all_words+=(str(comment).split())
    ##converting them to lowercase
    all_words=[word.lower() if word.isalpha() else word for word in all_words]
    ## creating a vocab which contains all the unique words
    vocabulary=list(set(all_words))

    train_set=pd.read_csv('train_set_preprocessed.csv')
    comm=train_set['0']
    ## creating a list of words 
    all_words=[]
    for comment in comm :
        all_words+=(str(comment).split())
    ##converting them to lowercase
    all_words=[word.lower() if word.isalpha() else word for word in all_words]
    vocabulary.extend(list(set(all_words)))
    vocabulary = list(set(vocabulary))
    return all_words, vocabulary

def unigram(all_words, vocabulary) :
    unigram_counts={}
    size=len(all_words)
    for word in all_words :
        if word in unigram_counts :
            unigram_counts[word]+=1
        else:
            unigram_counts[word]=1
    vocab_size=len(vocabulary)
    return unigram_counts, size, vocab_size

def calc_perplexity(unigram_counts, size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    ## in unigram model, probability of a sentence is product of probabilities of all the words
    perplexity=[]
    not_perplexable=[]
    for sentence in sentences :
        probability=1
        words_in_sent=0
        words=(str(sentence).split())
        words = [word.lower() if word.isalpha() else word for word in words]
        for word in words :
            if word not in unigram_counts :
                probability=0
            else :
                probability*=(unigram_counts[word])/(size)
            words_in_sent+=1
        ##calculating the value acc to the formula of perplexity
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))

            ##removing the sentences with very low probabilty, as perplex will be infinity
            if perplex>= 1000000:
                # print(probability,end=" ")
                # print(words_in_sent)
                not_perplexable.append(sentence)
                # pass
            else :
                perplexity.append(perplex)
                
        else :
            not_perplexable.append(sentence)
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity

all_words, vocabulary=vocab()
unigram_counts, size, vocab_size=unigram(all_words, vocabulary)



# print(unigram_counts)
# print(size)
average_perplexity, not_perplexable, perplexity=calc_perplexity(unigram_counts, size)
print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)


print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))





    
    


    


    


    


the average perplexity over all the sentences that are perplexable is  4619.325923758668
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 6387


In [84]:
## function for calculating the perplexity for unigram after laplace smoothing 
def calc_perplexity_smoothing(unigram_counts, size, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    ## in unigram model, probability of a sentence is product of probabilities of all the words
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    for sentence in sentences :
        probability=1
        perplex=1
        words=str(sentence).split()
        words = [word.lower() if word.isalpha() else word for word in words]
        words_in_sent=len(words)
        for word in words :
            if word not in unigram_counts :
                unigram_counts[word]=0
            probability*=((unigram_counts[word])+1)/(size+vocab_size)
            # print(probability)
            # prob.append(probability)
        ##calculating the value acc to the formula of perplexity
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 100000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

unigramcount_perplex, size, vocab_size=unigram(all_words, vocabulary)

average_perplexity, not_perplexable, perplexity, not_perplex=calc_perplexity_smoothing(unigramcount_perplex, size, vocab_size)
print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
print(not_perplex)
# print("the list of not perplexable sentences is :", not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex))
for x in not_perplexable:
    print(len(x))




the average perplexity over all the sentences that are perplexable is  13536.893234944624
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1e-323), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 7.33452586e-316), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 2.7201525e-316), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 3.845342e-317), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 2.27611068613e-313), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)

In [85]:
## function for calculating the perplexity for unigram after Add-k smoothing 
def calc_perplexity_smoothing_addk(unigram_counts, size, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    ## in unigram model, probability of a sentence is product of probabilities of all the words
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    k=0.1
    for sentence in sentences :
        probability=1
        perplex=1
        words=str(sentence).split()
        words = [word.lower() if word.isalpha() else word for word in words]
        words_in_sent=len(words)
        for word in words :
            if word not in unigram_counts :
                unigram_counts[word]=0
            probability*=((unigram_counts[word])+k)/(size+(vocab_size*k))
            # print(probability)
            # prob.append(probability)
        ##calculating the value acc to the formula of perplexity
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 100000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

unigramcount_perplex, size, vocab_size=unigram(all_words, vocabulary)

average_perplexity, not_perplexable, perplexity, not_perplex=calc_perplexity_smoothing_addk(unigramcount_perplex, size, vocab_size)
print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
print(not_perplex)
# print("the list of not perplexable sentences is :", not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex))
for x in not_perplexable:
    print(len(x))




the average perplexity over all the sentences that are perplexable is  61424.73265646102
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.5e-323), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.8480738e-316), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 9.524464e-316), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.186866644e-314), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1,

In [86]:
## training a bigram_model
## we alraedy created a necessary vocabulary, now we need to create a bigrams list, and bigram counts.

def create_bigram_list() :
    bigram_list=[]
    train_set=pd.read_csv('train_set_preprocessed.csv')
    comm=train_set['0']
    for comment in comm :
        words=(str(comment).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        for i in range(len(words)) :
            if i==0 :
                ## adding start of the sentence
                bigram_list.append(('<s>',words[i]))
            elif i==len(words)-1 :
                ## adding the end of the sentence 
                bigram_list.append((words[i],'</s>'))
            else :
                bigram_list.append((words[i], words[i+1]))
        size=len(bigram_list)
    return bigram_list, size

# print(create_bigram_list())

def count_bigrams(bigrams_list) :
    bigrams={}
    for bigram in bigrams_list :
        if bigram not in bigrams :
            bigrams[bigram]=1
        else :
            bigrams[bigram]+=1
    return bigrams

def calculate_perplexity_bigram(bigrams_count, unigram_counts) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    for sentence in sentences :
        words=(str(sentence).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        words_in_sent=len(words)
        for i in range(len(words)-1) :
            if (i==0) and (('<s>',words[i]) in bigrams_count):
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                probability*=(bigrams_count[('<s>',words[i])])/total_sentences
                if (words[i] in unigram_counts) and ((words[i],words[i+1]) in bigrams_count) :
                    probability*=(bigrams_count[(words[i], words[i+1])])/unigram_counts[words[i]]
            elif (i==len(words)-2) and ((words[i+1], '</s>') in bigrams_count) :
                ## asame logic as above for denominator 
                probability*=(bigrams_count[(words[i+1],'</s>')])/total_sentences
                if words[i] in unigram_counts and ((words[i],words[i+1]) in bigrams_count) :
                    probability*=(bigrams_count[(words[i], words[i+1])])/unigram_counts[words[i]]
            elif (words[i] in unigram_counts) and ((words[i],words[i+1]) in bigrams_count) :
                
                probability*=(bigrams_count[(words[i], words[i+1])])/unigram_counts[words[i]]
                
            else :
                probability=0
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 1000000:
                not_perplexable.append(sentence)
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity


bigram_list, size_of_bigrams= create_bigram_list()
bigrams_count=count_bigrams(bigram_list)
average_perplexity, not_perplexable, perplexity=calculate_perplexity_bigram(bigrams_count, unigram_counts)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)

print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))




        
        

    


    

the average perplexity over all the sentences that are perplexable is  441.22500263273145
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 35014


In [87]:
def calculate_perplexity_bigram_smoothing(bigrams_count, unigram_counts,vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    for sentence in sentences :
        words=str(sentence).split()
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        perplex=1
        words_in_sent=len(words)
        for i in range(len(words)-1) :
            
            if (i==0) :
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                if ('<s>', words[i]) not in bigrams_count :
                    bigrams_count[('<s>', words[i])]=0
            
                probability*=(bigrams_count[('<s>',words[i])]+1)/(total_sentences+vocab_size)
            
            elif (i==len(words)-2) :
                
                if (words[i+1], '</s>') not in bigrams_count :
                    bigrams_count[(words[i+1], '</s>')]=0
                 
                ## asame logic as above for denominator 
            
                probability*=(bigrams_count[(words[i+1],'</s>')]+1)/(total_sentences+vocab_size)
                if words[i] in unigram_counts  :
                    if (words[i],words[i+1]) not in bigrams_count:
                        bigrams_count[(words[i],words[i+1])]=0
                    probability*=(bigrams_count[(words[i], words[i+1])]+1)/(unigram_counts[words[i]]+vocab_size)
                    
                    
            if (words[i]) in unigram_counts  :
                if (words[i],words[i+1]) not in bigrams_count:
                    bigrams_count[(words[i],words[i+1])]=0
                    
            #### NOT TAKING INTO ACCOUNT WHEN THE FIRST DOES NOT EXIST AS WELL
            
                probability*=(bigrams_count[(words[i], words[i+1])]+1)/(unigram_counts[words[i]]+vocab_size)
            
            else :
                probability=0
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
                
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex


bigrams_count_perplex=count_bigrams(bigram_list)
average_perplexity, not_perplexable, perplexity, not_perplex_bigram=calculate_perplexity_bigram_smoothing(bigrams_count_perplex, unigram_counts,vocab_size)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)
print(not_perplex_bigram)
print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex_bigram))
for x in not_perplexable:
    print(len(x))


       
    

the average perplexity over all the sentences that are perplexable is  355839.73879125423
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0), (1, 0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (

In [88]:
def calculate_perplexity_bigram_smoothing_addk(bigrams_count, unigram_counts, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    k=0.1
    for sentence in sentences :
        words=str(sentence).split()
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        perplex=1
        words_in_sent=len(words)
        for i in range(len(words)-1) :
            
            if (i==0) :
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                if ('<s>', words[i]) not in bigrams_count :
                    bigrams_count[('<s>', words[i])]=0
            
                probability*=(bigrams_count[('<s>',words[i])]+k)/(total_sentences+(vocab_size*k))
            
            elif (i==len(words)-2) :
                
                if (words[i+1], '</s>') not in bigrams_count :
                    bigrams_count[(words[i+1], '</s>')]=0
                 
                ## asame logic as above for denominator 
            
                probability*=(bigrams_count[(words[i+1],'</s>')]+k)/(total_sentences+(vocab_size*k))
                if words[i] in unigram_counts  :
                    if (words[i],words[i+1]) not in bigrams_count:
                        bigrams_count[(words[i],words[i+1])]=0
                    probability*=(bigrams_count[(words[i], words[i+1])]+k)/(unigram_counts[words[i]]+(vocab_size*k))
                    
                    
            if (words[i]) in unigram_counts  :
                if (words[i],words[i+1]) not in bigrams_count:
                    bigrams_count[(words[i],words[i+1])]=0
                    
            #### NOT TAKING INTO ACCOUNT WHEN THE FIRST DOES NOT EXIST AS WELL
            
                probability*=(bigrams_count[(words[i], words[i+1])]+k)/(unigram_counts[words[i]]+(vocab_size*k))
            
            else :
                probability=0
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
                
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

bigrams_count_perplex=count_bigrams(bigram_list)

average_perplexity, not_perplexable, perplexity, not_perplex_bigram=calculate_perplexity_bigram_smoothing_addk(bigrams_count_perplex, unigram_counts, vocab_size)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)
print(not_perplex_bigram)
print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex_bigram))
for x in not_perplexable:
    print(len(x))


       
    

the average perplexity over all the sentences that are perplexable is  409646.94217110064
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0), (1, 0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0), (1, 0.0), (

In [89]:
def calculate_perplexity_bigram_smoothing_unigram_prior(bigrams_count, unigram_counts, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    m=100
    for sentence in sentences :
        words=str(sentence).split()
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        perplex=1
        words_in_sent=len(words)
        for i in range(len(words)-1) :
            
            if (i==0) :
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                if ('<s>', words[i]) not in bigrams_count :
                    bigrams_count[('<s>', words[i])]=0
            
                probability*=(bigrams_count[('<s>',words[i])]+(m*((unigram_counts[words[i]])+1)/(size+vocab_size)))/(total_sentences+m)
            
            elif (i==len(words)-2) :
                
                if (words[i+1], '</s>') not in bigrams_count :
                    bigrams_count[(words[i+1], '</s>')]=0
                 
                ## asame logic as above for denominator 
            
                probability*=(bigrams_count[(words[i+1],'</s>')]+(m*((total_sentences)+1)/(size+vocab_size)))/(total_sentences+m)
                if words[i] in unigram_counts  :
                    if (words[i],words[i+1]) not in bigrams_count:
                        bigrams_count[(words[i],words[i+1])]=0
                    probability*=(bigrams_count[(words[i], words[i+1])]+(m*((unigram_counts[words[i+1]])+1)/(size+vocab_size)))/(unigram_counts[words[i]]+m)
                    
                    
            if (words[i]) in unigram_counts  :
                if (words[i],words[i+1]) not in bigrams_count:
                    bigrams_count[(words[i],words[i+1])]=0
                    
            #### NOT TAKING INTO ACCOUNT WHEN THE FIRST DOES NOT EXIST AS WELL
            
                probability*=(bigrams_count[(words[i], words[i+1])]+(m*((unigram_counts[words[i+1]])+1)/(size+vocab_size)))/(unigram_counts[words[i]]+m)
            
            else :
                probability=0
        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
                
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

bigrams_count_perplex=count_bigrams(bigram_list)

average_perplexity, not_perplexable, perplexity, not_perplex_bigram=calculate_perplexity_bigram_smoothing_unigram_prior(bigrams_count_perplex, unigramcount_perplex, vocab_size)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)
print(not_perplex_bigram)
print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex_bigram))
for x in not_perplexable:
    print(len(x))


       
    

the average perplexity over all the sentences that are perplexable is  2675349.040623611
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (176964869317.24765, 1.8044249389362467e-34), (1, 0.0), (1, 0.0), (inf, 2.01193288551641e-309), (1, 0.0), (inf, 2.928184629744e-312), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 2.83447043e-315), (inf, 5.48006704374806e-309), (33201206278.023342, 2.732363097223394e-32), (49097917885.00591, 8.449106305071301e-33), (inf, 3.0262563e-317), (1, 0.0), (1, 0.0), (inf, 1.940517874174e-312), (1, 0.0), (1, 0.0), (27273433378.124, 4.929246757265931e-32), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 5.3949997e-317), (96211026883.63889, 1.1228595834571838e-33), (1, 0.0), (42396955799.864334, 1.3121877435574628e-32), (1, 0.0), (1, 0.0), (1, 0.0), (124331189692.60493, 5.2030707581783365e-34), (1, 0.0), (1, 0.0), (inf, 3.71

In [90]:
### trigram model , creating trigram list, trigram count
def create_trigram_list() :
    trigram_list=[]
    train_set=pd.read_csv('train_set_preprocessed.csv')
    comm=train_set['0']
    for comment in comm :
        words=(str(comment).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        for i in range(len(words)) :
            if i==0 :
                ## adding start of the sentence
                trigram_list.append(('<s>','<s>',words[i]))
            
            if i==1 :
                trigram_list.append(('<s>',words[0],words[1]))

            if i==len(words)-2 :
                trigram_list.append((words[i],words[i+1],'</s>'))

            if i==len(words)-1 :
                ## adding the end of the sentence 
                trigram_list.append((words[i],'</s>','</s>'))
                
            if (i!=(len(words)-1) and i!=(len(words)-2)) :
                trigram_list.append((words[i], words[i+1], words[i+2]))

        size=len(trigram_list)
    return trigram_list, size

trigram_list, trigram_size= create_trigram_list()
# print(trigram_size)

def count_trigram(trigram_list) :
    trigrams={}
    for trigram in trigram_list :
        if trigram not in trigrams :
            trigrams[trigram]=1
        else :
            trigrams[trigram]+=1
    return trigrams

trigrams_count=count_trigram(trigram_list)

def calculate_perplexity_trigram(trigrams_count,bigrams_count) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    for sentence in sentences :
        words=(str(sentence).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        words_in_sent=len(words)
        a=True
        for i in range(len(words)-2) :
            if (i==0) and (('<s>','<s>',words[i]) in trigrams_count):
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                probability*=(trigrams_count[('<s>','<s>',words[i])])/total_sentences
                a=False

            if (i==1) and (('<s>',words[0],words[1]) in trigrams_count) and (('<s>', words[0]) in bigrams_count):

                probability*=(trigrams_count[('<s>',words[0],words[1])])/bigrams_count[('<s>',words[0])]
                a=False

            if (i==len(words)-3) and ((words[i+1],words[i+2], '</s>') in trigrams_count) and ((words[i+2],'</s>','</s>') in trigrams_count) :
                
                ## asame logic as above for denominator 
                if (words[i+1],words[i+2]) in bigrams_count :
                    probability*=(trigrams_count[(words[i+1],words[i+2],'</s>')])/bigrams_count[(words[i+1],words[i+2])]
                if (words[i+2],'</s>') in bigrams_count :
                     probability*=(trigrams_count[(words[i+2],'</s>','</s>')])/bigrams_count[(words[i+2], '</s>')]
                a=False


            if ((words[i],words[i+1]) in bigrams_count) and ((words[i],words[i+1],words[i+2]) in trigrams_count) :

                probability*=(trigrams_count[(words[i], words[i+1], words[i+2])])/bigrams_count[(words[i], words[i+1])]
                a=False
            
            if a==True :
                probability=0

        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 1000000:
                not_perplexable.append(sentence)
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity

average_perplexity, not_perplexable, perplexity=calculate_perplexity_trigram(trigrams_count, bigrams_count)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)

print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))



    


the average perplexity over all the sentences that are perplexable is  9.576783993831928
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 1337


In [91]:
def calculate_perplexity_trigram_smoothing(trigrams_count,bigrams_count, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    for sentence in sentences :
        words=(str(sentence).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        perplex=1
        words_in_sent=len(words)
        a=True
        for i in range(len(words)-2) :
            if (i==0) :
                
                if ('<s>','<s>',words[i]) not in trigrams_count:
                    trigrams_count[('<s>','<s>',words[i])]=0
                
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                
                probability*=(trigrams_count[('<s>','<s>',words[i])]+1)/(total_sentences+vocab_size)
                a=False

            if (i==1) and (('<s>', words[0]) in bigrams_count):
              if  ('<s>',words[0],words[1]) not in trigrams_count :
                trigrams_count[('<s>',words[0],words[1])]=0
            
              probability*=(trigrams_count[('<s>',words[0],words[1])]+1)/(bigrams_count[('<s>',words[0])]+vocab_size)
              a=False

            if ((words[i],words[i+1]) in bigrams_count) :

                if (words[i],words[i+1],words[i+2]) not in trigrams_count:
                    trigrams_count[(words[i],words[i+1],words[i+2])]=0

                probability*=(trigrams_count[(words[i], words[i+1], words[i+2])]+1)/(bigrams_count[(words[i], words[i+1])]+vocab_size)
                a=False



            if (i==len(words)-3)  :
                
                ## asame logic as above for denominator 
                if (words[i+1],words[i+2]) in bigrams_count :
                    if (words[i+1],words[i+2], '</s>') not in trigrams_count :
                        trigrams_count[(words[i+1],words[i+2], '</s>')]=0
                    probability*=(trigrams_count[(words[i+1],words[i+2],'</s>')]+1)/(bigrams_count[(words[i+1],words[i+2])]+vocab_size)
                if (words[i+2],'</s>') in bigrams_count :
                     if (words[i+2],'</s>','</s>') not in trigrams_count :
                         trigrams_count[(words[i+2],'</s>','</s>')]=0
                     probability*=(trigrams_count[(words[i+2],'</s>','</s>')]+1)/(bigrams_count[(words[i+2], '</s>')]+vocab_size)
                a=False
            
            if a==True :
                probability=0

        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

average_perplexity, not_perplexable, perplexity, not_perplex_trigram=calculate_perplexity_trigram_smoothing(trigrams_count, bigrams_count, vocab_size)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)
print(not_perplex_trigram)
print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex_trigram))
for x in not_perplexable:
    print(len(x))



the average perplexity over all the sentences that are perplexable is  42432.27333820371
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 2.13748115305e-313), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.75215e-319), (inf, 1.037972109614e-312), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 2.336074714e-314), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 2.436e-321), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), 

In [92]:
def calculate_perplexity_trigram_smoothing_addk(trigrams_count,bigrams_count, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    k=0.1
    for sentence in sentences :
        words=(str(sentence).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        perplex=1
        words_in_sent=len(words)
        a=True
        for i in range(len(words)-2) :
            if (i==0) :
                
                if ('<s>','<s>',words[i]) not in trigrams_count:
                    trigrams_count[('<s>','<s>',words[i])]=0
                
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                
                probability*=(trigrams_count[('<s>','<s>',words[i])]+k)/(total_sentences+(vocab_size*k))
                a=False

            if (i==1) and (('<s>', words[0]) in bigrams_count):
              if  ('<s>',words[0],words[1]) not in trigrams_count :
                trigrams_count[('<s>',words[0],words[1])]=0
            
              probability*=(trigrams_count[('<s>',words[0],words[1])]+k)/(bigrams_count[('<s>',words[0])]+(vocab_size*k))
              a=False

            if ((words[i],words[i+1]) in bigrams_count) :

                if (words[i],words[i+1],words[i+2]) not in trigrams_count:
                    trigrams_count[(words[i],words[i+1],words[i+2])]=0

                probability*=(trigrams_count[(words[i], words[i+1], words[i+2])]+k)/(bigrams_count[(words[i], words[i+1])]+(vocab_size*k))
                a=False



            if (i==len(words)-3)  :
                
                ## asame logic as above for denominator 
                if (words[i+1],words[i+2]) in bigrams_count :
                    if (words[i+1],words[i+2], '</s>') not in trigrams_count :
                        trigrams_count[(words[i+1],words[i+2], '</s>')]=0
                    probability*=(trigrams_count[(words[i+1],words[i+2],'</s>')]+k)/(bigrams_count[(words[i+1],words[i+2])]+(vocab_size*k))
                if (words[i+2],'</s>') in bigrams_count :
                     if (words[i+2],'</s>','</s>') not in trigrams_count :
                         trigrams_count[(words[i+2],'</s>','</s>')]=0
                     probability*=(trigrams_count[(words[i+2],'</s>','</s>')]+k)/(bigrams_count[(words[i+2], '</s>')]+(vocab_size*k))
                a=False
            
            if a==True :
                probability=0

        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

average_perplexity, not_perplexable, perplexity, not_perplex_trigram=calculate_perplexity_trigram_smoothing_addk(trigrams_count, bigrams_count, vocab_size)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)
print(not_perplex_trigram)
print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex_trigram))
for x in not_perplexable:
    print(len(x))



the average perplexity over all the sentences that are perplexable is  9296.237097640602
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.1182e-319), (1, 0.0), (inf, 6.10131e-318), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 3.837574396e-315), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.592639315e-315), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.882e-320), (1, 0.0), (inf, 9.3334697263633e-311), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.519753014834e-311), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1,

In [93]:
def calculate_perplexity_trigram_smoothing_unigram_prior(trigrams_count,bigrams_count,unigram_counts, vocab_size) :
    test_set=pd.read_csv('test_set_preprocessed.csv')
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    m=100
    for sentence in sentences :
        words=(str(sentence).split())
        words=[word.lower() if word.isalpha() else word for word in words]
        probability=1
        perplex=1
        words_in_sent=len(words)
        a=True
        for i in range(len(words)-2) :
            if (i==0) :
                
                if ('<s>','<s>',words[i]) not in trigrams_count:
                    trigrams_count[('<s>','<s>',words[i])]=0
                
                ## as the denominator will be all sentences as start of sentence occurs in every sentence
                
                probability*=(trigrams_count[('<s>','<s>',words[i])]+(m*((unigram_counts[words[i]])+1)/(size+vocab_size)))/(total_sentences+m)
                a=False

            if (i==1) and (('<s>', words[0]) in bigrams_count):
              if  ('<s>',words[0],words[1]) not in trigrams_count :
                trigrams_count[('<s>',words[0],words[1])]=0
            
              probability*=(trigrams_count[('<s>',words[0],words[1])]+(m*((unigram_counts[words[1]])+1)/(size+vocab_size)))/(bigrams_count[('<s>',words[0])]+m)
              a=False

            if ((words[i],words[i+1]) in bigrams_count) :

                if (words[i],words[i+1],words[i+2]) not in trigrams_count:
                    trigrams_count[(words[i],words[i+1],words[i+2])]=0

                probability*=(trigrams_count[(words[i], words[i+1], words[i+2])]+(m*((unigram_counts[words[i+2]])+1)/(size+vocab_size)))/(bigrams_count[(words[i], words[i+1])]+m)
                a=False



            if (i==len(words)-3)  :
                
                ## asame logic as above for denominator 
                if (words[i+1],words[i+2]) in bigrams_count :
                    if (words[i+1],words[i+2], '</s>') not in trigrams_count :
                        trigrams_count[(words[i+1],words[i+2], '</s>')]=0
                    probability*=(trigrams_count[(words[i+1],words[i+2],'</s>')]+(m*(total_sentences+1)/(size+vocab_size)))/(bigrams_count[(words[i+1],words[i+2])]+m)
                if (words[i+2],'</s>') in bigrams_count :
                     if (words[i+2],'</s>','</s>') not in trigrams_count :
                         trigrams_count[(words[i+2],'</s>','</s>')]=0
                     probability*=(trigrams_count[(words[i+2],'</s>','</s>')]+(m*(total_sentences+1)/(size+vocab_size)))/(bigrams_count[(words[i+2], '</s>')]+m)
                a=False
            
            if a==True :
                probability=0

        if probability!=0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            # perplexity.append(perplex)
            if perplex>= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

average_perplexity, not_perplexable, perplexity, not_perplex_trigram=calculate_perplexity_trigram_smoothing_unigram_prior(trigrams_count, bigrams_count,unigramcount_perplex, vocab_size)

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
# print(perplexity)
print(not_perplex_trigram)
print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
print(len(not_perplex_trigram))
for x in not_perplexable:
    print(len(x))



the average perplexity over all the sentences that are perplexable is  293.1831486301467
[(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 7.98438053e-316), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 6e-323), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 4.975926765e-314), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (inf, 1.12046818578807e-310), (1, 0.0), (inf, 5e-324), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]
['they mostly come from rpakistan pakistani nationalists are a cancera general message to everyoneplease raise awareness about a recent terrifying rape case that happened in pakistan a day ago about 400 men raped and assaulted a woman in lahore punjab in open public in front of the minar e pakistan our version

In [94]:
def create_quadgram_list() :
    quadgram_list=[]
    train_set=pd.read_csv("train_set_preprocessed.csv")
    comm=train_set['0']
    #words = []
    for comment in comm :
        words=(str(comment).split())
        if(len(words) < 3): continue
        for i in range(len(words)) :
            if i == 0:
                ## adding start of the sentence
                quadgram_list.append(('<s>','<s>','<s>',words[i]))
            if i == 1:
                quadgram_list.append(('<s>','<s>',words[i - 1],words[i]))
            if i == 2:
                quadgram_list.append(('<s>',words[i - 2],words[i - 1],words[i]))
            if i == len(words) - 3:
                ## adding the end of the sentence
                quadgram_list.append((words[i],words[i+1],words[i+2],'</s>'))
            if i == len(words) - 2:
                quadgram_list.append((words[i],words[i+1],'</s>','</s>'))
            if i == len(words) - 1:
                quadgram_list.append((words[i],'</s>','</s>','</s>'))
            if (i != (len(words) - 1) and i != (len(words) - 2) and i != (len(words) - 3)):
                quadgram_list.append((words[i], words[i + 1],words[i + 2],words[i + 3]))
    size=len(quadgram_list)
    return quadgram_list, size



def count_quadgrams(quadgrams_list) :
    quadgrams={}
    for quadgram in quadgrams_list :
        if quadgram not in quadgrams :
            quadgrams[quadgram] = 1
        else :
            quadgrams[quadgram] += 1
    return quadgrams



def calculate_quadgram_perplexity(quadgrams_count,trigrams_count) :
    test_set=pd.read_csv("test_set_preprocessed.csv")
    sentences=test_set['0']
    total_sentences=len(sentences) # number of <s>
    perplexity=[]
    not_perplexable=[]
    #perplexable_cnt = 0
    #perplexity=0
    #not_perplexable=0
    for sentence in sentences :
        words=(str(sentence).split())
        probability=1
        words_in_sent=len(words)
        if(words_in_sent < 1):
            not_perplexable.append(sentence)
            continue
        for i in range(len(words)-2) :
            chk = 0
            if ((i==0) and (('<s>','<s>','<s>',words[i]) in quadgrams_count)):
                probability *= (quadgrams_count[('<s>','<s>','<s>',words[i])])/total_sentences
                chk = 1
            if ((i==1) and (('<s>','<s>',words[i - 1],words[i]) in quadgrams_count) and (('<s>','<s>',words[i - 1]) in trigrams_count)):
                probability *= (quadgrams_count[('<s>','<s>',words[i-1],words[i])])/trigrams_count[('<s>','<s>',words[i - 1])]
                chk = 1
            if ((i==2) and (('<s>',words[i-2],words[i - 1],words[i]) in quadgrams_count) and (('<s>',words[i-2],words[i - 1]) in trigrams_count)):
                probability *= (quadgrams_count[('<s>',words[i-2],words[i-1],words[i])])/trigrams_count[('<s>',words[i-2],words[i - 1])]
                chk = 1
            if (i==len(words)-3) and ((words[i],words[i+1],words[i+2],'</s>') in quadgrams_count and ((words[i+1],words[i+2],'</s>','</s>') in quadgrams_count) and ((words[i+2],'</s>','</s>','</s>') in quadgrams_count)):
                if ((words[i],words[i+1],words[i+2]) in trigrams_count):
                    probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')])/trigrams_count[(words[i],words[i+1],words[i+2])]
                    chk = 1
                if ((words[i+1],words[i+2],'</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')])/trigrams_count[(words[i+1],words[i+2],'</s>')]
                    chk = 1
                if ((words[i+2],'</s>','</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+2],'</s>','</s>','</s>')])/trigrams_count[(words[i+2],'</s>','</s>')]
                    chk = 1
            if ((i != len(words)-3) and ((words[i],words[i+1],words[i+2]) in trigrams_count) and ((words[i],words[i+1],words[i+2],words[i+3]) in quadgrams_count)) :
                probability *=(quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])])/trigrams_count[(words[i],words[i+1],words[i+2])]
                chk = 1
            if chk == 0:
                probability=0
        
        if probability != 0 :
            perplex=(1/(probability))**(1/(words_in_sent))
            #perplexity.append(perplex)
            if perplex >= 1000000:
                not_perplexable.append(sentence)
                #not_perplexable += 1
            else :
                perplexity.append(perplex)
                #perplexity += perplex
                #perplexable_cnt += 1
        else :
            not_perplexable.append(sentence)
            #not_perplexable += 1
    average_perplexity=sum(perplexity)/len(perplexity)
    #average_perplexity = perplexity/perplexable_cnt
    return average_perplexity, not_perplexable, perplexity

quadgrams_list, quadgrams_size  = create_quadgram_list()
'''for i in range(0,10):
    print(quadgrams_list[i])'''
quadgrams = count_quadgrams(quadgrams_list)
'''for i in range(0,10):
    print(quadgrams_list[i], quadgrams[quadgrams_list[i]])'''
average_perplexity, not_perplexable, perplexity = calculate_quadgram_perplexity(quadgrams,trigrams_count)
print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
print("the total sentences in test set are ", len(perplexity) + len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))


##OUTPUT
#the average perplexity over all the sentences that are perplexable is  5.475069992222899
#the total sentences in test set are  67870
#the total no of not_perplexable sentences in the validation set are : 44635

#####################################################################################################################################

the average perplexity over all the sentences that are perplexable is  5.637616496767182
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 44551


In [95]:
#WITH SMOOTHING
def calculate_perplexity_quadgram_smoothing(quadgrams_count,trigrams_count, vocab_size) :
    test_set=pd.read_csv("test_set_preprocessed.csv")
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    for sentence in sentences :
        words=(str(sentence).split())
        probability=1
        perplex=1
        words_in_sent=len(words)
        if(words_in_sent < 1):
            not_perplexable.append(sentence)
            continue
        for i in range(len(words)-2) :
            chk = 0
            if (i == 0):
                if (('<s>','<s>','<s>',words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>','<s>','<s>',words[i])] = 0
                probability *= (quadgrams_count[('<s>','<s>','<s>',words[i])] + 1)/(total_sentences + vocab_size)
                chk = 1
            if ((i==1) and (('<s>','<s>',words[i - 1]) in trigrams_count)):
                if (('<s>','<s>',words[i - 1],words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>','<s>',words[i - 1],words[i])] = 0
                probability *= (quadgrams_count[('<s>','<s>',words[i-1],words[i])] + 1)/(trigrams_count[('<s>','<s>',words[i - 1])] + vocab_size)
                chk = 1
            if ((i==2) and (('<s>',words[i-2],words[i - 1]) in trigrams_count)):
                if (('<s>',words[i-2],words[i - 1],words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>',words[i-2],words[i - 1],words[i])] = 0
                probability *= (quadgrams_count[('<s>',words[i-2],words[i-1],words[i])] + 1)/(trigrams_count[('<s>',words[i-2],words[i - 1])] + vocab_size)
                chk = 1
            if (i==len(words)-3):
                if ((words[i],words[i+1],words[i+2],'</s>') not in quadgrams_count):
                    quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')] = 0
                if ((words[i+1],words[i+2],'</s>','</s>') not in quadgrams_count):
                    quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')] = 0
                if ((words[i+2],'</s>','</s>','</s>') not in quadgrams_count):
                    quadgrams_count[(words[i+2],'</s>','</s>','</s>')] = 0
                if ((words[i],words[i+1],words[i+2]) in trigrams_count):
                    probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')] + 1)/(trigrams_count[(words[i],words[i+1],words[i+2])] + vocab_size)
                    chk = 1
                if ((words[i+1],words[i+2],'</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')] + 1)/(trigrams_count[(words[i+1],words[i+2],'</s>')] + vocab_size)
                    chk = 1
                if ((words[i+2],'</s>','</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+2],'</s>','</s>','</s>')] + 1)/(trigrams_count[(words[i+2],'</s>','</s>')] + vocab_size)
                    chk = 1
            if ((i != len(words)-3) and ((words[i],words[i+1],words[i+2]) in trigrams_count)):
                if ((words[i],words[i+1],words[i+2],words[i+3]) not in quadgrams_count):
                    quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])] = 0
                probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])] + 1)/(trigrams_count[(words[i],words[i+1],words[i+2])] + vocab_size)
                chk = 1
            if chk == 0:
                probability=0

        if probability != 0:
            perplex=(1/(probability))**(1/(words_in_sent))
            #perplexity.append(perplex)
            if perplex >= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
            
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

average_perplexity, not_perplexable, perplexity, not_perplex_quadgram=calculate_perplexity_quadgram_smoothing(quadgrams,trigrams_count,len(vocabulary))

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
##print(not_perplex_trigram)
##print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
##print(len(not_perplex_trigram))
##for x in not_perplexable:
##    print(len(x))

##OUTPUT
#the average perplexity over all the sentences that are perplexable is  285189.33577858837
#the total sentences in test set are  67870
#the total no of not_perplexable sentences in the validation set are : 38199



the average perplexity over all the sentences that are perplexable is  417417.8508132651
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 27727


In [96]:
#WITH SMOOTHING
def calculate_perplexity_quadgram_smoothing_addk(quadgrams_count,trigrams_count, vocab_size) :
    test_set=pd.read_csv("test_set_preprocessed.csv")
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    k=0.01
    for sentence in sentences :
        words=(str(sentence).split())
        probability=1
        perplex=1
        words_in_sent=len(words)
        if(words_in_sent < 1):
            not_perplexable.append(sentence)
            continue
        for i in range(len(words)-2) :
            chk = 0
            if (i == 0):
                if (('<s>','<s>','<s>',words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>','<s>','<s>',words[i])] = 0
                probability *= (quadgrams_count[('<s>','<s>','<s>',words[i])] + k)/(total_sentences + (vocab_size*k))
                chk = 1
            if ((i==1) and (('<s>','<s>',words[i - 1]) in trigrams_count)):
                if (('<s>','<s>',words[i - 1],words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>','<s>',words[i - 1],words[i])] = 0
                probability *= (quadgrams_count[('<s>','<s>',words[i-1],words[i])] + k)/(trigrams_count[('<s>','<s>',words[i - 1])] + (vocab_size*k))
                chk = 1
            if ((i==2) and (('<s>',words[i-2],words[i - 1]) in trigrams_count)):
                if (('<s>',words[i-2],words[i - 1],words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>',words[i-2],words[i - 1],words[i])] = 0
                probability *= (quadgrams_count[('<s>',words[i-2],words[i-1],words[i])] + k)/(trigrams_count[('<s>',words[i-2],words[i - 1])] + (vocab_size*k))
                chk = 1
            if (i==len(words)-3):
                if ((words[i],words[i+1],words[i+2],'</s>') not in quadgrams_count):
                    quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')] = 0
                if ((words[i+1],words[i+2],'</s>','</s>') not in quadgrams_count):
                    quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')] = 0
                if ((words[i+2],'</s>','</s>','</s>') not in quadgrams_count):
                    quadgrams_count[(words[i+2],'</s>','</s>','</s>')] = 0
                if ((words[i],words[i+1],words[i+2]) in trigrams_count):
                    probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')] + k)/(trigrams_count[(words[i],words[i+1],words[i+2])] + (vocab_size*k))
                    chk = 1
                if ((words[i+1],words[i+2],'</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')] + k)/(trigrams_count[(words[i+1],words[i+2],'</s>')] + (vocab_size*k))
                    chk = 1
                if ((words[i+2],'</s>','</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+2],'</s>','</s>','</s>')] + k)/(trigrams_count[(words[i+2],'</s>','</s>')] + (vocab_size*k))
                    chk = 1
            if ((i != len(words)-3) and ((words[i],words[i+1],words[i+2]) in trigrams_count)):
                if ((words[i],words[i+1],words[i+2],words[i+3]) not in quadgrams_count):
                    quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])] = 0
                probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])] + k)/(trigrams_count[(words[i],words[i+1],words[i+2])] + (vocab_size*k))
                chk = 1
            if chk == 0:
                probability=0

        if probability != 0:
            perplex=(1/(probability))**(1/(words_in_sent))
            #perplexity.append(perplex)
            if perplex >= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
            
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

average_perplexity, not_perplexable, perplexity, not_perplex_quadgram=calculate_perplexity_quadgram_smoothing_addk(quadgrams,trigrams_count,len(vocabulary))

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
##print(not_perplex_trigram)
##print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
##print(len(not_perplex_trigram))
##for x in not_perplexable:
##    print(len(x))

##OUTPUT
#the average perplexity over all the sentences that are perplexable is  285189.33577858837
#the total sentences in test set are  67870
#the total no of not_perplexable sentences in the validation set are : 38199



the average perplexity over all the sentences that are perplexable is  41311.67056764389
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 27699


In [97]:
#WITH SMOOTHING
def calculate_perplexity_quadgram_smoothing_unigram_prior(quadgrams_count,trigrams_count,unigram_counts, vocab_size) :
    test_set=pd.read_csv("test_set_preprocessed.csv")
    sentences=test_set['0']
    total_sentences=len(sentences)
    perplexity=[]
    not_perplexable=[]
    not_perplex=[]
    m=10
    for sentence in sentences :
        words=(str(sentence).split())
        probability=1
        perplex=1
        words_in_sent=len(words)
        if(words_in_sent < 1):
            not_perplexable.append(sentence)
            continue
        for i in range(len(words)-2) :
            chk = 0
            if (i == 0):
                if (('<s>','<s>','<s>',words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>','<s>','<s>',words[i])] = 0
                probability *= (quadgrams_count[('<s>','<s>','<s>',words[i])] + (m*((unigram_counts[words[i]])+1)/(size+vocab_size)))/(total_sentences + m)
                chk = 1
            if ((i==1) and (('<s>','<s>',words[i - 1]) in trigrams_count)):
                if (('<s>','<s>',words[i - 1],words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>','<s>',words[i - 1],words[i])] = 0
                probability *= (quadgrams_count[('<s>','<s>',words[i-1],words[i])] + (m*((unigram_counts[words[i]])+1)/(size+vocab_size)))/(trigrams_count[('<s>','<s>',words[i - 1])] + m)
                chk = 1
            if ((i==2) and (('<s>',words[i-2],words[i - 1]) in trigrams_count)):
                if (('<s>',words[i-2],words[i - 1],words[i]) not in quadgrams_count):
                    quadgrams_count[('<s>',words[i-2],words[i - 1],words[i])] = 0
                probability *= (quadgrams_count[('<s>',words[i-2],words[i-1],words[i])] + (m*((unigram_counts[words[i]])+1)/(size+vocab_size)))/(trigrams_count[('<s>',words[i-2],words[i - 1])] + m)
                chk = 1
            if (i==len(words)-3):
                if ((words[i],words[i+1],words[i+2],'</s>') not in quadgrams_count):
                    quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')] = 0
                if ((words[i+1],words[i+2],'</s>','</s>') not in quadgrams_count):
                    quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')] = 0
                if ((words[i+2],'</s>','</s>','</s>') not in quadgrams_count):
                    quadgrams_count[(words[i+2],'</s>','</s>','</s>')] = 0
                if ((words[i],words[i+1],words[i+2]) in trigrams_count):
                    probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],'</s>')] + (m*(total_sentences+1)/(size+vocab_size)))/(trigrams_count[(words[i],words[i+1],words[i+2])] + m)
                    chk = 1
                if ((words[i+1],words[i+2],'</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+1],words[i+2],'</s>','</s>')] + (m*(total_sentences+1)/(size+vocab_size)))/(trigrams_count[(words[i+1],words[i+2],'</s>')] + m)
                    chk = 1
                if ((words[i+2],'</s>','</s>') in trigrams_count):
                    probability *= (quadgrams_count[(words[i+2],'</s>','</s>','</s>')] + (m*(total_sentences+1)/(size+vocab_size)))/(trigrams_count[(words[i+2],'</s>','</s>')] + m)
                    chk = 1
            if ((i != len(words)-3) and ((words[i],words[i+1],words[i+2]) in trigrams_count)):
                if ((words[i],words[i+1],words[i+2],words[i+3]) not in quadgrams_count):
                    quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])] = 0
                probability *= (quadgrams_count[(words[i],words[i+1],words[i+2],words[i+3])] + (m*((unigram_counts[words[i+3]])+1)/(size+vocab_size)))/(trigrams_count[(words[i],words[i+1],words[i+2])] + m)
                chk = 1
            if chk == 0:
                probability=0

        if probability != 0:
            perplex=(1/(probability))**(1/(words_in_sent))
            #perplexity.append(perplex)
            if perplex >= 10000000000:
                not_perplexable.append(sentence)
                not_perplex.append((perplex,probability))
            else :
                perplexity.append(perplex)
        else :
            not_perplexable.append(sentence)
            not_perplex.append((perplex,probability))
            
    average_perplexity=sum(perplexity)/len(perplexity)
    return average_perplexity, not_perplexable, perplexity, not_perplex

average_perplexity, not_perplexable, perplexity, not_perplex_quadgram=calculate_perplexity_quadgram_smoothing_unigram_prior(quadgrams,trigrams_count,unigramcount_perplex,len(vocabulary))

print("the average perplexity over all the sentences that are perplexable is ", average_perplexity)
##print(not_perplex_trigram)
##print(not_perplexable)
print("the total sentences in test set are ", len(perplexity)+ len(not_perplexable))
print("the total no of not_perplexable sentences in the validation set are :", len(not_perplexable))
##print(len(not_perplex_trigram))
##for x in not_perplexable:
##    print(len(x))

##OUTPUT
#the average perplexity over all the sentences that are perplexable is  285189.33577858837
#the total sentences in test set are  67870
#the total no of not_perplexable sentences in the validation set are : 38199



the average perplexity over all the sentences that are perplexable is  611.9278905129706
the total sentences in test set are  66563
the total no of not_perplexable sentences in the validation set are : 27690
